In [ ]:
!pip install langchain_openai
!pip install langchain-community
!pip install pypdf
!pip install faiss-cpu
!pip install pandas langchain openai

In [ ]:
import pandas as pd

# 1. 사용 환경 준비

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

OpenAI API key 입력: ··········


# 2. 모델 초기화

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")

# 3. 문서 로드

In [ ]:
from langchain.document_loaders import PyPDFLoader

#  파일 로드. 파일의 경로 입력

loader_culture = CSVLoader("final(culture).csv")
loader_place = CSVLoader("final(place).csv")
# 페이지 별 문서 로드
docs_cul = loader_culture.load()
docs_pla = loader_place.load()


In [ ]:
print(docs_cul)
print(len(docs_cul))
print(docs_pla)
print(len(docs_pla))

In [ ]:
# chunk

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs_cul)

print(splits[0])

page_content='번호: 1호
명칭: 서울 숭례문
소재지: 서울 중구 세종대로 40 (남대문로4가)
지정일: 1962년12월 20일지정' metadata={'source': 'final(culture).csv', 'row': 0}


In [ ]:
# Embedding 작업
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002") # 토큰화된 문서를 모델에 입력하여 임베딩 벡터를 생성하고, 이를 평균하여 전체 문서의 벡터를 생성

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 7. Retriever 객체 생성


In [ ]:
#Retriever 객체 생성
from langchain.vectorstores.base import VectorStore

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# 8. 프롬프트 템플릿 정의

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

# 9. RAG 체인 구성

In [ ]:
# 디버깅을 위해 만든 클래스
class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

# 프롬프트 클래스
class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template

    def invoke(self, inputs):
        # response_docs 내용을 trim해줌 (가독성을 높여줌)
        if isinstance(inputs, list): # inputs가 list인 경우. 즉 여러개의 문서들이 검색되어 리스트로 전달된 경우
            context_text = "\n".join([doc.page_content for doc in inputs]) # \n을 구분자로 넣어서 한 문자열로 합쳐줌
        else:
            context_text = inputs # 리스트가 아닌경우는 그냥 리턴해줌

        # 프롬프트
        formatted_prompt = self.prompt_template.format_messages( # 템플릿의 변수에 삽입해줌
            context=context_text, # {context} 변수에 context_text, 즉 검색된 문서 내용을 삽입함
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever 클래스
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        # 0단계 : query의 타입에 따른 전처리
        if isinstance(inputs, dict): # inputs가 딕셔너리 타입일경우, question 키의 값을 검색 쿼리로 사용
            query = inputs.get("question", "")
        else: # 질문이 문자열로 주어지면, 그대로 검색 쿼리로 사용
            query = inputs
        # 1단계 : query를 리트리버에 넣어주고, response_docs를 얻어모
        response_docs = self.retriever.get_relevant_documents(query) # 검색을 수행하고 검색 결과를 response_docs에 저장
        return response_docs

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever), # 클래스 객체를 생성해서 value로 넣어줌
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

# 10. 챗봇 구동 확인

In [ ]:
# 챗봇 구동
while True:
    print("========================")

    # 0. 질문을 받아서 query에 저장함
    query = input("질문을 입력하세요 : ")

    # 1. 리트리버로 question에 대한 검색 결과를 response_docs에 저장함
    response_docs = rag_chain_debug["context"].invoke({"question": query})

    # 2. 프롬프트에 질문과 response_docs를 넣어줌
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })

    # 3. 완성된 프롬프트를 LLM에 넣어줌
    response = rag_chain_debug["llm"].invoke(prompt_messages)

    print("\n답변:")
    print(response.content)

<ipython-input-39-3d4ee348912e>:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_docs = self.retriever.get_relevant_documents(query) # 검색을 수행하고 검색 결과를 response_docs에 저장



답변:
culture의 1호는 '번호: 11-1호'입니다.

답변:
숭례문은 1호야.

답변:
조선방역지도는 248호야.

답변:
숭례문 근처 여행지로는 국립중앙박물관이 있습니다.

답변:
1. 코리아나화장박물관 (서울 강남구 언주로 827) - 지정일: 2004년 6월 26일
2. 성신여대박물관 (서울 성북구 보문로34다길 2) - 지정일: 1985년 8월 9일
3. 국립중앙박물관 (서울 서초구 서빙고로 137) - 지정일: 1986년 11월 29일
4. 세종대왕기념관 (서울 동대문구 회기로 56) - 지정일: 1984년 5월 30일


## 회고
1. 처음에는 CSV 파일 각각 chunk를 해서 합치려고 공식문서의 merge 기능을 사용해 봤으나 실패해서 다시 처음으로 돌아가 각각 chunk를 하였다.
2. 답변만 작성되어서, 팀(프로젝트)과 소통을 한 뒤, 질문을 남겨서 보여줄 것인지, 답만 확인하는 것을 사용할 지 정해야할 것 같다.
3. 작동은 하나 부족한 면이 많다. 특히 반말과 존댓말이 공존해서 수정이 필요하다(프롬프트 추가로 가능한지)
4. 이외에도 수정할 점들을 하나씩 풀어나가야겠다.